In [1]:
from transformers import BertTokenizer
import numpy as np
import onnxruntime as ort
from joblib import load

tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased')

text = "Murata Manufacturing Co., Ltd.,,homekit,TRADFRI gateway,TRADFRI gateway,"

inputs = tokenizer("Text: " + text, return_tensors="pt", padding='max_length', truncation=True, max_length=128)

# convert to numpy array
input_ids = inputs["input_ids"].numpy()
attention_mask = inputs["attention_mask"].numpy()
attention_mask_float = attention_mask.astype(np.float32)

In [2]:
# ONNX model
onnx_model_path = './onnx/model.onnx'

# create ONNX Runtime
session = ort.InferenceSession(onnx_model_path)

# run ONNX model
outputs = session.run(None, {"input_ids": input_ids, "attention_mask": attention_mask_float})

# extract logits
logits = outputs[0]

# predict
predictions = np.argmax(logits, axis=-1)

# load LabelEncoder
labelencoder = load('labelencoder.joblib')

# decode
predicted_label = labelencoder.inverse_transform(predictions)[0]
print(f"Predicted output label is: {predicted_label}")

Predicted output label is: IKEA
